In [2]:
import os
import pandas as pd
import numpy as np
import preprocessor as prep
import postprocessor as postp
import processor as proc
import pandas as pd
import unifloc.uniflocpy.uTools.plotly_workflow as pw
import datetime
from sklearn import metrics
import multiprocessing as mp
from multiprocessing import Pool
from preprocessor import load_and_edit_chess_data


from sklearn import metrics
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.decomposition import PCA
from pandas.api.types import is_numeric_dtype

C:\Users\LemikhovAlex\Documents\CET\git\unifloc_vba\\


Немножко пошалим, тупо запустим все скважины, про которые есть `%well_name%.csv` чтобы протестить работу с шахматками. Не надо забывать закрывать всплывающие вкладки, они убьют компуцкер

In [ ]:
well_names = ['569', '601', '1354', '1567', '1602', '1628']
for well_name in well_names:
    chess_file_name = 'Скв. ' + well_name + ' (01.08.2018-28.02.2019).xls'
    tr_name = "Техрежим, , февраль 2019.xls"
    # TODO убрать флаги
    read_initial_data = True
    plot_initial_data = True
    create_input_data = True
    multiprocessing_on = True #TODO может быть убрать флаги, как лучше?
    created_input_data_type = 0 # 0 - adaptation input, 1 - restore input with calibation coefficients

    current_path = os.getcwd()
    time_mark = datetime.datetime.today().strftime('%Y_%m_%d_%H_%M_%S')
    path_to_data = current_path + "\\data\\"
    path_to_work_dir = current_path + "\\data\\" + well_name +  "\\"
    save_dir_name = 'init_edit'
    path_to_save = path_to_work_dir + save_dir_name + '\\'

    cs_data_filename = path_to_save + well_name + "_first_edit.csv"

    left_boundary = [datetime.datetime(2019,1,1)]#, datetime.datetime(2018,11,29)]
    right_boundary = [datetime.datetime(2019,2,28)]#, datetime.datetime(2019,2,28)]


    if read_initial_data:
        try:
            os.mkdir(path_to_work_dir + save_dir_name)
        except:
            pass
        data_file_path = path_to_work_dir + well_name + ".csv"
        try:
            try:
                well_data = pd.read_csv(data_file_path,sep=';', header=None, skiprows = 1)
            except:
                well_data = pd.read_csv(data_file_path,sep='\\t', header=None)
            well_data = prep.initial_editing(well_data, well_name)
            well_data = prep.create_edited_df(well_data)
        except:
            data_file_path = path_to_work_dir + well_name + ".csv"
            well_data = pd.read_csv(data_file_path, sep='\\t', skiprows = 1, skipfooter =1, index_col = 'Названия строк')#, header=None)
            well_data.index.name = 'Время'
            well_data.index = pd.to_datetime(well_data.index)
            del well_data['Общий итог']
            well_data.columns = prep.del_inf_in_columns_name(well_data, well_name)

    #TODO использовать библиотеку modin вместо pandas
    well_data.to_csv(path_to_save + well_name + "_first_edit.csv")

    well_data['ГФ, м3/м3'] = well_data['Объемный дебит газа'] / well_data['Объемный дебит нефти']
    qliq = {'Объемный дебит жидкости':['Объемный дебит жидкости']}
    gor = {'ГФ, м3/м3':['ГФ, м3/м3']}
    wc = {'Процент обводненности':['Процент обводненности']}
    pressure_intake = {'Давление на приеме': ['Давление на приеме насоса (пласт. жидкость)']}
    pressure_wh = {'Линейное давление': ['Линейное давление']}
    temp_intake = {'Температура на приеме насоса': ['Температура на приеме насоса (пласт. жидкость)']}
    frequencies = {'Выходная частота ПЧ':
                   ['Выходная частота ПЧ']}
    load = {'Загрузка двигателя': ['Загрузка двигателя']}
    power = {'Активная мощность':['Активная мощность']}
    voltage = {'Напряжение на выходе ТМПН':['Напряжение на выходе ТМПН']}
    cos = {'Коэффициент мощности':['Коэффициент мощности']}
    time = {'Время замеров':['Время замера фактическое','Время замера плановое']}
    current =  {'Ток фазы А':['Ток фазы А']}
    all_banches = [qliq,gor, wc, time, pressure_intake, pressure_wh, 
                   temp_intake, frequencies, load,power, voltage,cos,current]

    pw.create_report_html(well_data, all_banches, path_to_save + well_name + "_first_edit_report.html")
    del well_data['ГФ, м3/м3']

    chess_data = prep.load_and_edit_chess_data(path_to_work_dir + chess_file_name,
                                         '1d', without_changing = True) #TODO сделать чтение исходной шахматки с рус. датами
    qliq = {'Qж ТМ':['Qж ТМ']}
    wc = {'Обв ТМ':['Обв ТМ']}
    pressure_intake = {'Рэцн ТМ': ['Рэцн ТМ']}
    pressure_wh = {'Рлин ТМ': ['Рлин ТМ']}
    pressure_buf = {'Р буферное': ['Рбуф ТМ', 'Рбуф']}
    temp_intake = {'Темп. ж. ТМ ': ['Темп. ж. ТМ']}
    frequencies = {'F вращ ТМ':
                   ['F вращ ТМ']}
    load = {'Загр. ПЭД (ТМ)': ['Загр. ПЭД (ТМ)']}
    cos = {'Cos ф (ТМ)':['Cos ф (ТМ)']}
    choke = {'Dшт':['Dшт']}
    current =  {'I A ТМ':['I A ТМ']}
    all_banches = [qliq, wc, pressure_intake, pressure_wh, pressure_buf,
                   temp_intake, frequencies, load,cos,choke, current]

    pw.create_report_html(chess_data, all_banches, path_to_save + well_name + "_chess_report.html")
    print(f'{well_name} done')

Потратим времени на обработку выбросов

In [51]:
well_name = '569'
chess_file_name = 'Скв. ' + well_name + ' (01.08.2018-28.02.2019).xls'
    
tr_name = "Техрежим, , февраль 2019.xls"
# TODO убрать флаги
read_initial_data = True
plot_initial_data = True
create_input_data = True
multiprocessing_on = True #TODO может быть убрать флаги, как лучше?
created_input_data_type = 0 # 0 - adaptation input, 1 - restore input with calibation coefficients

current_path = os.getcwd()
time_mark = datetime.datetime.today().strftime('%Y_%m_%d_%H_%M_%S')
path_to_data = current_path + "\\data\\"
path_to_work_dir = current_path + "\\data\\" + well_name +  "\\"
save_dir_name = 'init_edit'
path_to_save = path_to_work_dir + save_dir_name + '\\'

cs_data_filename = path_to_save + well_name + "_first_edit.csv"

left_boundary = [datetime.datetime(2019,1,1)]#, datetime.datetime(2018,11,29)]
right_boundary = [datetime.datetime(2019,2,28)]#, datetime.datetime(2019,2,28)]


if read_initial_data:
    try:
        os.mkdir(path_to_work_dir + save_dir_name)
    except:
        pass
    data_file_path = path_to_work_dir + well_name + ".csv"
    try:
        try:
            well_data = pd.read_csv(data_file_path,sep=';', header=None, skiprows = 1)
        except:
            well_data = pd.read_csv(data_file_path,sep='\\t', header=None)
        well_data = prep.initial_editing(well_data, well_name)
        well_data = prep.create_edited_df(well_data)
    except:
        data_file_path = path_to_work_dir + well_name + ".csv"
        well_data = pd.read_csv(data_file_path, sep='\\t', skiprows = 1, skipfooter =1, index_col = 'Названия строк')#, header=None)
        well_data.index.name = 'Время'
        well_data.index = pd.to_datetime(well_data.index)
        del well_data['Общий итог']
        well_data.columns = prep.del_inf_in_columns_name(well_data, well_name)

#TODO использовать библиотеку modin вместо pandas
well_data.to_csv(path_to_save + well_name + "_first_edit.csv")

well_data['ГФ, м3/м3'] = well_data['Объемный дебит газа'] / well_data['Объемный дебит нефти']
qliq = {'Объемный дебит жидкости':['Объемный дебит жидкости']}
gor = {'ГФ, м3/м3':['ГФ, м3/м3']}
wc = {'Процент обводненности':['Процент обводненности']}
pressure_intake = {'Давление на приеме': ['Давление на приеме насоса (пласт. жидкость)']}
pressure_wh = {'Линейное давление': ['Линейное давление']}
temp_intake = {'Температура на приеме насоса': ['Температура на приеме насоса (пласт. жидкость)']}
frequencies = {'Выходная частота ПЧ':
                ['Выходная частота ПЧ']}
load = {'Загрузка двигателя': ['Загрузка двигателя']}
power = {'Активная мощность':['Активная мощность']}
voltage = {'Напряжение на выходе ТМПН':['Напряжение на выходе ТМПН']}
cos = {'Коэффициент мощности':['Коэффициент мощности']}
time = {'Время замеров':['Время замера фактическое','Время замера плановое']}
current =  {'Ток фазы А':['Ток фазы А']}
all_banches = [qliq,gor, wc, time, pressure_intake, pressure_wh, 
               temp_intake, frequencies, load,power, voltage,cos,current]

pw.create_report_html(well_data, all_banches, path_to_save + well_name + "_first_edit_report.html")
del well_data['ГФ, м3/м3']

chess_data = prep.load_and_edit_chess_data(path_to_work_dir + chess_file_name,
                                         '1d', without_changing = True) #TODO сделать чтение исходной шахматки с рус. датами
qliq = {'Qж ТМ':['Qж ТМ']}
wc = {'Обв ТМ':['Обв ТМ']}
pressure_intake = {'Рэцн ТМ': ['Рэцн ТМ']}
pressure_wh = {'Рлин ТМ': ['Рлин ТМ']}
pressure_buf = {'Р буферное': ['Рбуф ТМ', 'Рбуф']}
temp_intake = {'Темп. ж. ТМ ': ['Темп. ж. ТМ']}
frequencies = {'F вращ ТМ':
                ['F вращ ТМ']}
load = {'Загр. ПЭД (ТМ)': ['Загр. ПЭД (ТМ)']}
cos = {'Cos ф (ТМ)':['Cos ф (ТМ)']}
choke = {'Dшт':['Dшт']}
current =  {'I A ТМ':['I A ТМ']}
all_banches = [qliq, wc, pressure_intake, pressure_wh, pressure_buf,
               temp_intake, frequencies, load,cos,choke, current
              ]

pw.create_report_html(chess_data, all_banches, path_to_save + well_name + "_chess_report.html")
print(f'{well_name} done')

569 done


In [52]:
time_to_resamle = '3h'
created_input_data_type = 0 

In [53]:
%%time
edited_data_cs = prep.load_and_edit_cs_data(cs_data_filename,
                                       time_to_resamle, 
                                       created_input_data_type)

Wall time: 1.4 s


In [54]:
chess_data = prep.load_and_edit_chess_data(path_to_work_dir + chess_file_name,
                                     time_to_resamle)

In [55]:
input_data_dir_name = 'adapt_input_' + time_mark
path_to_input_data = path_to_work_dir + input_data_dir_name + '\\'
if create_input_data:
    try:
        os.mkdir(path_to_work_dir + input_data_dir_name)
    except:
        pass
created_input_data = edited_data_cs.join(chess_data, how = 'outer')
created_input_data = created_input_data.dropna(subset = ['Объемный дебит жидкости (СУ)'])
created_input_data = prep.cut_df(created_input_data, left_boundary, right_boundary)
plot_file_path = path_to_input_data + well_name + '_adapt_input.html'
input_data_traces = pw.create_traces_list_for_all_columms(created_input_data, 'lines+markers', use_gl = True)
pw.plot_subplots(input_data_traces, plot_file_path, True)

created_input_data.to_csv(path_to_input_data + well_name + '_adapt_input.csv')

In [56]:
created_input_data['Линейное давление (СУ)'] =  created_input_data['Линейное давление (СУ)'] * 10
qliq = {'Объемный дебит жидкости (СУ)':['Объемный дебит жидкости (СУ)']}
pressure_intake = {'Давление на приеме': ['Давление на приеме насоса (пласт. жидкость) (СУ)']}
pressure_wh = {'Рлин ТМ (Ш)': ['Рлин ТМ (Ш)', 'Линейное давление (СУ)']}
pressure_bf = {'Рбуф (Ш)': ['Рбуф (Ш)']}
temp_intake = {'Температура на приеме насоса (пласт. жидкость) (СУ)': ['Температура на приеме насоса (пласт. жидкость) (СУ)']}
frequencies = {'Частота, Гц':
               ['F вращ ТМ (Ш)', 'Выходная частота ПЧ (СУ)']}
choke = {'Размер штуцера, мм': ['Dшт (Ш)']}
power = {'Активная мощность (СУ)':['Активная мощность (СУ)']}
voltage = {'Напряжение на выходе ТМПН (СУ)':['Напряжение на выходе ТМПН (СУ)']}
cos = {'Коэффициент мощности (СУ)':['Коэффициент мощности (СУ)']}
gor = {'ГФ (СУ)':['ГФ (СУ)']}
wc = {'Процент обводненности (СУ)':['Процент обводненности (СУ)']}
all_banches = [qliq, gor,wc, pressure_intake, pressure_wh, pressure_bf,
               temp_intake, frequencies, choke,power,voltage,cos]

pw.create_report_html(created_input_data, all_banches, path_to_input_data + well_name + '_adapt_input_report.html')

# Адаптация
Нужно, используя сгенерированные данные `well_name_adapt_input` получить значения коэффициентов калибровок по напору и мощности с помощью `processor.py`

Для этого выключить флаги

* vfm_calc_option = False
* restore_q_liq_only = False

и определить папку с входными данными `dir_name_with_input_data`

In [57]:
dir_name_with_input_data = 'adaptation_2019_11_26_13_54'

In [58]:
dir_name_with_input_data = 'adapt_input_2019_11_26_13_40_21' + '\\'
input_data_file_name = well_name + '_adapt_input'
dir_name_with_calculated_data = 'adaptation_2019_11_26_13_54' + '\\'
calculated_data_file_name = well_name + '_adapt_1'

In [59]:
if multiprocessing_on == True:
    filenames_list = []
    for (dirpath, dirnames, filenames) in os.walk(path_to_work_dir + dir_name_with_calculated_data + 'multiprocessing\\'):
        filenames_list.extend(filenames)
        break
    print(filenames_list)
    for i,j in enumerate(filenames_list):
        if i == 0:
            first_result_data = pd.read_csv(path_to_work_dir + dir_name_with_calculated_data + 'multiprocessing\\' + j , parse_dates = True, index_col = 'Время')
        else:
            another_result_data = pd.read_csv(path_to_work_dir + dir_name_with_calculated_data + 'multiprocessing\\' + j , parse_dates = True, index_col = 'Время')
            first_result_data = first_result_data.append(another_result_data, sort = True)
            del first_result_data['d']
            first_result_data = first_result_data.dropna(subset = ['ESP.ESPpump.EffiencyESP_d'])
    first_result_data.to_csv(path_to_work_dir + dir_name_with_calculated_data + calculated_data_file_name + '.csv')

['569_adapt_1.csv', '569_adapt_2.csv', '569_adapt_3.csv', '569_adapt_4.csv', '569_adapt_5.csv', '569_adapt_6.csv', '569_adapt_7.csv', '569_adapt_8.csv']


In [60]:
calculated_data = prep.load_calculated_data_from_csv(path_to_work_dir + dir_name_with_calculated_data +
                                                calculated_data_file_name +  '.csv')

In [61]:
input_data = pd.read_csv(path_to_work_dir + dir_name_with_input_data + input_data_file_name +  '.csv')
input_data.index = input_data['Время']
del input_data['Время']
all_data = input_data.join(calculated_data, how = 'outer')

all_data.to_csv(path_to_work_dir + dir_name_with_calculated_data + well_name + '_calc_and_input' +  '.csv' )

In [62]:
adapt_data_traces = pw.create_traces_list_for_all_columms(all_data, 'lines+markers', use_gl = True)
plot_file_path = path_to_work_dir + dir_name_with_calculated_data + well_name + '_calc_and_input' +  '.html'
pw.plot_subplots(adapt_data_traces, plot_file_path, True)

In [63]:
qliq = {'Объемный дебит жидкости':['Объемный дебит жидкости (СУ)', 'Q ж, м3/сут (Модель)']}
calibrs = {'Калибровки по напору и мощности':
       ['К. калибровки по напору - множитель (Модель)',
           'К. калибровки по мощности - множитель (Модель)']}
metrics = {'Метрики расчета':
       ['Значение функции ошибки (Модель)']}
gor = {'ГФ (Модель)':['ГФ (Модель)']}
wc = {'Обводненность, %':['Обв, % (Модель)']}
pressure_intake = {'Давление на приеме': ['Давление на приеме насоса (пласт. жидкость) (СУ)', 'P прием ЭЦН, атм (Модель)']}
pressure_wh = {'Рлин': ['Рлин ТМ (Ш)', 'P лин., атм (Модель)']}
pressure_bf = {'Рбуф': ['Рбуф (Ш)', 'P буф., атм (Модель)']}
temp_intake = {'Температура на приеме насоса': ['Температура на приеме насоса (пласт. жидкость) (СУ)',
                                                                      'T прием ЭЦН, C (Модель)']}
frequencies = {'Частота, Гц':
               ['F вращ ТМ (Ш)', 'Выходная частота ПЧ (СУ)', 'F тока, ГЦ (Модель)']}
choke = {'Размер штуцера, мм': ['Dшт (Ш)']}
power = {'Активная мощность (СУ)':['Активная мощность (СУ)']}
true_power = {'Мощность, передаваемая СУ (Модель)':['Мощность, передаваемая СУ (Модель)']}
voltage = {'Напряжение на выходе ТМПН (СУ)':['Напряжение на выходе ТМПН (СУ)']}
cos = {'Коэффициент мощности (СУ)':['Коэффициент мощности (СУ)']}

efficiency = {'КПД ЭЦН, д.ед.':
          ['КПД ЭЦН, д.ед. (Модель)']}
dif_pressure = {'Перепад давления в ЭЦН, атм':
            ['Перепад давления в ЭЦН, атм (Модель)']}
all_banches = [qliq,calibrs,metrics, gor,wc, pressure_intake, pressure_wh, pressure_bf,
               temp_intake, frequencies, choke,power,true_power, voltage,cos, efficiency, dif_pressure]

pw.create_report_html(all_data, all_banches, path_to_work_dir + dir_name_with_calculated_data + 
                      well_name + '_adapt_report.html')

Убедились, что теперь `processor.py` разбирает все даные на обработку

Дальше занимаемся исследованием аномалий, обусловленых малым временем замера

In [109]:
d_anomaly = pd.read_csv(f'data/{well_name}/init_edit/{well_name}_first_edit.csv')

In [110]:
for c in d_anomaly.columns.values:
    print(c)
d_anomaly.describe()

Время
Динамический уровень
Частота турбинного вращения
Процент обводненности
Массовый дебит нефти
Массовый дебит жидкости
Давление на выкиде насоса
Расход жидкости
Плотность жидкости
Температура на выкиде насоса
Объемный дебит газа
Объемный дебит нефти
Объемный дебит жидкости
Объемный дебит воды
Активная энергия (прямая) за весь период
Активная мощность
Общее количество запусков насоса
Количество качаний в минуту
Активная энергия (обратная) за весь период
Реактивная энергия (обратная) за весь период
Выходная частота ПЧ
Ток фазы А
Ток фазы В
Ток фазы С
Дисбаланс токов
Ток в звене постоянного тока
Дисбаланс тока полуходовой
Коэффициент мощности
Нагрузка на шток максимальная
Нагрузка на шток минимальная
Линейное давление
Полная мощность
Давление на приеме насоса (масло двигателя)
Температура на приеме насоса (масло двигателя)
Режимный дебит газа
Режимный дебит жидкости
Реактивная энергия (прямая) за весь период
Связь с ТМС (1-да, 0-нет)
Температура обмоток двигателя
Давление на приеме нас

,Динамический уровень,Частота турбинного вращения,Процент обводненности,Массовый дебит нефти,Массовый дебит жидкости,Давление на выкиде насоса,Расход жидкости,Плотность жидкости,Температура на выкиде насоса,Объемный дебит газа,...,Виброускорение (результирующий вектор),Вибрация насоса по оси X,Вибрация насоса по оси Y,Входное напряжение АВ,Входное напряжение ВС,Входное напряжение СА,Дисбаланс напряжений,Загрузка двигателя,Затрубное давление,Общее время работы насоса
count,12.0,283.000000,346.000000,348.000000,348.000000,12.0,12.0,12.0,12.0,348.000000,...,11.0,11.0,11.0,100379.000000,102256.000000,104178.000000,15.000000,224741.000000,11.0,3131.000000
mean,0.0,21.077739,33.400462,224.309368,249.231926,0.0,0.0,945.0,0.0,7981.488345,...,0.0,0.0,0.0,392.014266,394.241472,393.512239,0.133333,71.965178,0.0,2083.491536
std,0.0,23.231359,10.449118,31.127556,34.585702,0.0,0.0,0.0,0.0,1690.220444,...,0.0,0.0,0.0,1.848405,1.743050,1.498952,0.351866,7.491019,0.0,1180.530803
min,0.0,0.000000,23.709999,76.919998,85.470001,0.0,0.0,945.0,0.0,1084.380005,...,0.0,0.0,0.0,385.000000,388.000000,387.000000,0.000000,0.000000,0.0,308.000000
25%,0.0,0.000000,29.062500,213.547501,237.272499,0.0,0.0,945.0,0.0,7163.705078,...,0.0,0.0,0.0,391.000000,393.000000,393.000000,0.000000,69.000000,0.0,1090.500000
50%,0.0,0.000000,29.400000,231.050003,256.720001,0.0,0.0,945.0,0.0,8372.769531,...,0.0,0.0,0.0,392.000000,394.000000,394.000000,0.000000,73.000000,0.0,1873.000000
75%,0.0,46.000000,37.759998,234.972496,261.080002,0.0,0.0,945.0,0.0,8818.604736,...,0.0,0.0,0.0,393.000000,396.000000,395.000000,0.000000,76.000000,0.0,2968.000000
max,0.0,48.000000,100.000000,443.140015,492.369995,0.0,0.0,945.0,0.0,20037.900391,...,0.0,0.0,0.0,398.000000,401.000000,399.000000,1.000000,105.000000,0.0,4573.000000


In [ ]:
pl_df = d_anomaly[['Время', 'Объемный дебит жидкости']].dropna
plt.plot(pl_df['Время'], pl_df['Объемный дебит жидкости'])

KeyboardInterrupt: 